In [111]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

examples = [
    {
        "question": "Express the movie 'Top Gun' with three emojis that come to your mind",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question":  "Express the movie 'The Godfather' with three emojis that come to your mind",
        "answer": "👨‍👨‍👦🔫🍝",
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]
    
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="""
    Previous conversation:
    {history}
    Human: Express the movie '{movie}' with three emojis that come to your mind
    """,
    input_variables=["movie"],
)


# 기존 코드는 유지하고 memory 부분을 추가합니다
memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

# chain 정의를 수정합니다
chain = (
    RunnablePassthrough.assign(
        history=load_memory
    ) 
    | prompt 
    | chat
)



# invoke_chain 함수 추가
def invoke_chain(movie_title):
    result = chain.invoke({"movie": movie_title})
    
    # 대화 내용을 memory에 저장
    memory.save_context(
        {"input": f"Tell me about {movie_title}"},
        {"output": result.content}
    )
    
    print(result)
    return result

# 사용 예시
invoke_chain("Avatar")

content='AI: 🌿🌌👽'


AIMessage(content='AI: 🌿🌌👽')

In [112]:
free_prompt = PromptTemplate.from_template("""
    Previous conversation:
    {history}
    Human: {question}
""")

free_chain = RunnablePassthrough.assign(history=load_memory) | free_prompt | chat
free_chain.invoke({"question": "이전에 질문한 영화가 무엇이지?"})

AIMessage(content='AI: 이전에 질문한 영화는 "Avatar"입니다.')